In [ ]:
from functools import partial
from fvhoe.boundary_conditions import BoundaryCondition
from fvhoe.hydro import advection_dt
from fvhoe.initial_conditions import sedov_blast
from fvhoe.solver import EulerSolver
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
N = 64
p = 3
solver = EulerSolver(
    w0=partial(sedov_blast, radius=0.05, Pmin_Pmax=(1, 2)),
    bc=BoundaryCondition(x="free", y="free"),
    nx=N,
    ny=N,
    px=p,
    py=p,
    riemann_solver="hllc",
    a_posteriori_slope_limiting=True,
    slope_limiter="minmod",
    cupy=True,
)

In [ ]:
solver.rkorder(0.2, [0.05, 0.1, 0.15])

In [ ]:
fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)

for var, label, idx in zip(
    ["rho", "P", "vx", "vy"],
    [r"$\overline{\rho}$", r"$\overline{P}$", r"$\overline{v}_x$", r"$\overline{v}_y$"],
    [(0, 0), (0, 1), (1, 0), (1, 1)],
):
    im = solver.plot_2d_slice(ax[idx], t=1, param=var, cmap="gnuplot", z=0.5)
    fig.colorbar(im, ax=ax[idx], label=label)

ax[0, 0].set_ylabel("$y$")
ax[1, 0].set_ylabel("$y$")
ax[1, 0].set_xlabel("$x$")
ax[1, 1].set_xlabel("$x$")

In [ ]:
for n in [0, -1]:
    m_tot = np.sum(solver.snapshots[0]["fv"].rho[:, 0, 0]) * (
        solver.h[0] * solver.h[1] * solver.h[2]
    )
    print(f"Total mass at time {solver.snapshot_times[n]}: {m_tot}")

In [ ]:
print(f"min rho at latest snapshot: {np.min(solver.snapshots[-1]['fv'].rho)}")
print(f"max rho at latest snapshot: {np.max(solver.snapshots[-1]['fv'].rho)}")